In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [10]:
r = requests.get('https://myanimelist.net/anime/genre/1/Action?page=1')
soup = BeautifulSoup(r.text, 'html.parser')

In [20]:
soup.find('h3',class_='h3_anime_subtitle')

In [12]:
soup.find_all('a',class_='link-title')

[<a class="link-title" href="https://myanimelist.net/anime/16498/Shingeki_no_Kyojin">Shingeki no Kyojin</a>,
 <a class="link-title" href="https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood">Fullmetal Alchemist: Brotherhood</a>,
 <a class="link-title" href="https://myanimelist.net/anime/30276/One_Punch_Man">One Punch Man</a>,
 <a class="link-title" href="https://myanimelist.net/anime/11757/Sword_Art_Online">Sword Art Online</a>,
 <a class="link-title" href="https://myanimelist.net/anime/31964/Boku_no_Hero_Academia">Boku no Hero Academia</a>,
 <a class="link-title" href="https://myanimelist.net/anime/38000/Kimetsu_no_Yaiba">Kimetsu no Yaiba</a>,
 <a class="link-title" href="https://myanimelist.net/anime/20/Naruto">Naruto</a>,
 <a class="link-title" href="https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011">Hunter x Hunter (2011)</a>,
 <a class="link-title" href="https://myanimelist.net/anime/22319/Tokyo_Ghoul">Tokyo Ghoul</a>,
 <a class="link-title" href="https://my

In [14]:
product_links = set()

for x in range(1, 5):
    r = requests.get(f'https://myanimelist.net/anime/genre/1/Action?page={x}')
    soup = BeautifulSoup(r.text, 'html.parser')
    anime_links = soup.find_all('a', class_='link-title')

    for link in anime_links:
        title = link.text
        url = link['href']
        product_links.add((title, url))
        print(f"Title: {title}\nLink: {url}\n")

product_links = list(product_links)

Title: Shingeki no Kyojin
Link: https://myanimelist.net/anime/16498/Shingeki_no_Kyojin

Title: Fullmetal Alchemist: Brotherhood
Link: https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood

Title: One Punch Man
Link: https://myanimelist.net/anime/30276/One_Punch_Man

Title: Sword Art Online
Link: https://myanimelist.net/anime/11757/Sword_Art_Online

Title: Boku no Hero Academia
Link: https://myanimelist.net/anime/31964/Boku_no_Hero_Academia

Title: Kimetsu no Yaiba
Link: https://myanimelist.net/anime/38000/Kimetsu_no_Yaiba

Title: Naruto
Link: https://myanimelist.net/anime/20/Naruto

Title: Hunter x Hunter (2011)
Link: https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011

Title: Tokyo Ghoul
Link: https://myanimelist.net/anime/22319/Tokyo_Ghoul

Title: Shingeki no Kyojin Season 2
Link: https://myanimelist.net/anime/25777/Shingeki_no_Kyojin_Season_2

Title: Jujutsu Kaisen
Link: https://myanimelist.net/anime/40748/Jujutsu_Kaisen

Title: Boku no Hero Academia 2nd Season
Li

In [36]:
r = requests.get('https://myanimelist.net/anime/11757/Sword_Art_Online')
soup = BeautifulSoup(r.text, 'html.parser')

In [47]:
soup.find_all('span',class_='dark_text')

[<span class="dark_text">Synonyms:</span>,
 <span class="dark_text">Japanese:</span>,
 <span class="dark_text">English:</span>,
 <span class="dark_text">Type:</span>,
 <span class="dark_text">Episodes:</span>,
 <span class="dark_text">Status:</span>,
 <span class="dark_text">Aired:</span>,
 <span class="dark_text">Premiered:</span>,
 <span class="dark_text">Broadcast:</span>,
 <span class="dark_text">Producers:</span>,
 <span class="dark_text">Licensors:</span>,
 <span class="dark_text">Studios:</span>,
 <span class="dark_text">Source:</span>,
 <span class="dark_text">Genres:</span>,
 <span class="dark_text">Themes:</span>,
 <span class="dark_text">Duration:</span>,
 <span class="dark_text">Rating:</span>,
 <span class="dark_text">Score:</span>,
 <span class="dark_text">Ranked:</span>,
 <span class="dark_text">Popularity:</span>,
 <span class="dark_text">Members:</span>,
 <span class="dark_text">Favorites:</span>]

In [56]:
r = requests.get('https://myanimelist.net/anime/11757/Sword_Art_Online')
soup = BeautifulSoup(r.text, 'html.parser')

divs = soup.find_all('div', class_='spaceit_pad')
anime_info = {}

for div in divs:
    span = div.find('span', class_='dark_text')
    if span:
        label = span.get_text(strip=True)
        value = div.get_text(strip=True, separator=' ')[len(label) + 1:].strip()
        # Fix Genres and Themes
        if label == 'Genres:':
            value = [genre.strip() for genre in value.split(',')]
        elif label == 'Themes:':
            value = [theme.strip() for theme in value.split(',')]
        anime_info[label] = value

for key, value in anime_info.items():
    print(f"{key}: {', '.join(value) if isinstance(value, list) else value}")



Synonyms:: S.A.O, SAO
Japanese:: ソードアート・オンライン
English:: Sword Art Online
Type:: TV
Episodes:: 25
Status:: Finished Airing
Aired:: Jul 8, 2012 to Dec 23, 2012
Premiered:: Summer 2012
Broadcast:: Sundays at 00:00 (JST)
Producers:: Aniplex , Genco , DAX Production , ASCII Media Works , Bandai Namco Games
Licensors:: Aniplex of America
Studios:: A-1 Pictures
Source:: Light novel
Genres:: Action Action, Adventure Adventure, Fantasy Fantasy, Romance Romance
Themes:: Love Polygon Love Polygon, Video Game Video Game
Duration:: 23 min. per ep.
Rating:: PG-13 - Teens 13 or older
Score:: 7.20 1 (scored by 2137462 2,137,462 users) 1 indicates a weighted score .
Ranked:: #3183 2 2 based on the top anime page. Please note that 'Not yet aired' and 'R18+' titles are excluded.
Popularity:: #5
Members:: 3,055,714
Favorites:: 68,496


In [58]:
def extract_anime_info(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    divs = soup.find_all('div', class_='spaceit_pad')
    anime_info = {}

    for div in divs:
        span = div.find('span', class_='dark_text')
        if span:
            label = span.get_text(strip=True)
            value = div.get_text(strip=True, separator=' ')[len(label) + 1:].strip()
            # Fix Genres and Themes
            if label == 'Genres:':
                value = [genre.strip() for genre in value.split(',')]
            elif label == 'Themes:':
                value = [theme.strip() for theme in value.split(',')]
            anime_info[label] = value

    return anime_info

product_links = set()

for x in range(1, 2):
    r = requests.get(f'https://myanimelist.net/anime/genre/1/Action?page={x}')
    soup = BeautifulSoup(r.text, 'html.parser')
    anime_links = soup.find_all('a', class_='link-title')

    for link in anime_links:
        title = link.text
        url = link['href']
        product_links.add((title, url))

product_links = list(product_links)

for title, url in product_links:
    anime_info = extract_anime_info(url)
    print(f"Title: {title}")
    for key, value in anime_info.items():
        print(f"{key}: {', '.join(value) if isinstance(value, list) else value}")
    print()


Title: Sword Art Online
Synonyms:: S.A.O, SAO
Japanese:: ソードアート・オンライン
English:: Sword Art Online
Type:: TV
Episodes:: 25
Status:: Finished Airing
Aired:: Jul 8, 2012 to Dec 23, 2012
Premiered:: Summer 2012
Broadcast:: Sundays at 00:00 (JST)
Producers:: Aniplex , Genco , DAX Production , ASCII Media Works , Bandai Namco Games
Licensors:: Aniplex of America
Studios:: A-1 Pictures
Source:: Light novel
Genres:: Action Action, Adventure Adventure, Fantasy Fantasy, Romance Romance
Themes:: Love Polygon Love Polygon, Video Game Video Game
Duration:: 23 min. per ep.
Rating:: PG-13 - Teens 13 or older
Score:: 7.20 1 (scored by 2137462 2,137,462 users) 1 indicates a weighted score .
Ranked:: #3183 2 2 based on the top anime page. Please note that 'Not yet aired' and 'R18+' titles are excluded.
Popularity:: #5
Members:: 3,055,714
Favorites:: 68,496

Title: One Punch Man
Synonyms:: One Punch-Man, One-Punch Man, OPM
Japanese:: ワンパンマン
English:: One Punch Man
Type:: TV
Episodes:: 12
Status:: Finished

In [64]:
def extract_anime_info(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    divs = soup.find_all('div', class_='spaceit_pad')
    anime_info = {}

    for div in divs:
        span = div.find('span', class_='dark_text')
        if span:
            label = span.get_text(strip=True)
            value = div.get_text(strip=True, separator=' ')[len(label) + 1:].strip()
            # Fix Genres and Themes
            if label == 'Genres:':
                value = ', '.join([genre.strip() for genre in value.split(',')])
            elif label == 'Themes:':
                value = ', '.join([theme.strip() for theme in value.split(',')])
            anime_info[label[:-1]] = value  # Remove the trailing colon

    return anime_info

product_links = []

for x in range(1, 2):  #Modify the number of pages here 
    r = requests.get(f'https://myanimelist.net/anime/genre/1/Action?page={x}')
    soup = BeautifulSoup(r.text, 'html.parser')
    anime_links = soup.find_all('a', class_='link-title')

    for link in anime_links:
        title = link.text
        url = link['href']
        product_links.append((title, url))

anime_data = []

for title, url in product_links:
    anime_info = extract_anime_info(url)
    anime_info['Title'] = title
    anime_data.append(anime_info)

df = pd.DataFrame(anime_data)

columns_order = ['Title', 'Synonyms', 'Japanese', 'English', 'Type', 'Episodes', 'Status', 'Aired', 
                 'Premiered', 'Broadcast', 'Producers', 'Licensors', 'Studios', 'Source', 'Genres', 
                 'Themes', 'Demographic', 'Duration', 'Rating', 'Score', 'Ranked', 'Popularity', 
                 'Members', 'Favorites']

df = df.reindex(columns=columns_order)

In [65]:
df.to_csv('anime.csv')

In [66]:
df

,Title,Synonyms,Japanese,English,Type,Episodes,Status,Aired,Premiered,Broadcast,...,Genres,Themes,Demographic,Duration,Rating,Score,Ranked,Popularity,Members,Favorites
0,Shingeki no Kyojin,"AoT, SnK",進撃の巨人,Attack on Titan,TV,25,Finished Airing,"Apr 7, 2013 to Sep 29, 2013",Spring 2013,Sundays at 01:58 (JST),...,"Action Action, Award Winning Award Winning, Dr...","Gore Gore, Military Military, Survival Survival",Shounen Shounen,24 min. per ep.,R - 17+ (violence & profanity),"8.54 1 (scored by 2778396 2,778,396 users) 1 i...",#109 2 2 based on the top anime page. Please n...,#1,"3,918,102","171,055"
1,Fullmetal Alchemist: Brotherhood,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",鋼の錬金術師 FULLMETAL ALCHEMIST,Fullmetal Alchemist: Brotherhood,TV,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,Sundays at 17:00 (JST),...,"Action Action, Adventure Adventure, Drama Dram...",NaN,Shounen Shounen,24 min. per ep.,R - 17+ (violence & profanity),"9.09 1 (scored by 2106269 2,106,269 users) 1 i...",#2 2 2 based on the top anime page. Please not...,#3,"3,321,679","224,722"
2,One Punch Man,"One Punch-Man, One-Punch Man, OPM",ワンパンマン,One Punch Man,TV,12,Finished Airing,"Oct 5, 2015 to Dec 21, 2015",Fall 2015,Mondays at 01:05 (JST),...,"Action Action, Comedy Comedy","Adult Cast Adult Cast, Parody Parody, Super Po...",Seinen Seinen,24 min. per ep.,R - 17+ (violence & profanity),"8.50 1 (scored by 2220532 2,220,532 users) 1 i...",#129 2 2 based on the top anime page. Please n...,#4,"3,195,626","64,214"
3,Sword Art Online,"S.A.O, SAO",ソードアート・オンライン,Sword Art Online,TV,25,Finished Airing,"Jul 8, 2012 to Dec 23, 2012",Summer 2012,Sundays at 00:00 (JST),...,"Action Action, Adventure Adventure, Fantasy Fa...","Love Polygon Love Polygon, Video Game Video Game",NaN,23 min. per ep.,PG-13 - Teens 13 or older,"7.20 1 (scored by 2137462 2,137,462 users) 1 i...",#3183 2 2 based on the top anime page. Please ...,#5,"3,055,714","68,496"
4,Boku no Hero Academia,NaN,僕のヒーローアカデミア,My Hero Academia,TV,13,Finished Airing,"Apr 3, 2016 to Jun 26, 2016",Spring 2016,Sundays at 17:00 (JST),...,NaN,"School School, Super Power Super Power",Shounen Shounen,24 min. per ep.,PG-13 - Teens 13 or older,"7.87 1 (scored by 2053916 2,053,916 users) 1 i...",#830 2 2 based on the top anime page. Please n...,#6,"3,001,386","53,507"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Rakudai Kishi no Cavalry,"A Chivalry of the Failed Knight, Rakudai Kishi...",落第騎士の英雄譚《キャバルリィ》,Chivalry of a Failed Knight,TV,12,Finished Airing,"Oct 3, 2015 to Dec 19, 2015",Fall 2015,Saturdays at 23:00 (JST),...,"Action Action, Fantasy Fantasy, Romance Romanc...",NaN,NaN,23 min. per ep.,R - 17+ (violence & profanity),"7.43 1 (scored by 555054 555,054 users) 1 indi...",#2077 2 2 based on the top anime page. Please ...,#162,"945,816","8,778"
96,Darker than Black: Kuro no Keiyakusha,DTB,Darker than BLACK -黒の契約者-,Darker than Black,TV,25,Finished Airing,"Apr 6, 2007 to Sep 28, 2007",Spring 2007,Fridays at 01:25 (JST),...,"Action Action, Mystery Mystery, Sci-Fi Sci-Fi",NaN,NaN,24 min. per ep.,R - 17+ (violence & profanity),"8.05 1 (scored by 436431 436,431 users) 1 indi...",#554 2 2 based on the top anime page. Please n...,#163,"943,718","16,714"
97,Tensei shitara Slime Datta Ken 2nd Season,Tensura 2,転生したらスライムだった件,That Time I Got Reincarnated as a Slime Season 2,TV,12,Finished Airing,"Jan 12, 2021 to Mar 30, 2021",Winter 2021,Tuesdays at 23:00 (JST),...,"Action Action, Adventure Adventure, Comedy Com...","Isekai Isekai, Reincarnation Reincarnation",Shounen Shounen,23 min. per ep.,PG-13 - Teens 13 or older,"8.38 1 (scored by 544930 544,930 users) 1 indi...",#190 2 2 based on the top anime page. Please n...,#164,"943,104","10,697"
98,Fate/stay night,Fate - Stay Night,Fate/stay night,Fate/stay night,TV,24,Finished Airing,"Jan 7, 2006 to Jun 17, 2006",Winter 2006,Saturdays at 01:30 (JST),...,"Action Action, Fantasy Fantasy, Romance Romanc...",NaN,NaN,24 min. per ep.,

Adventure

In [3]:
def extract_anime_info(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    divs = soup.find_all('div', class_='spaceit_pad')
    anime_info = {}

    for div in divs:
        span = div.find('span', class_='dark_text')
        if span:
            label = span.get_text(strip=True)
            value = div.get_text(strip=True, separator=' ')[len(label) + 1:].strip()
            if label == 'Genres:':
                value = ', '.join([genre.strip() for genre in value.split(',')])
            elif label == 'Themes:':
                value = ', '.join([theme.strip() for theme in value.split(',')])
            anime_info[label[:-1]] = value  

    return anime_info

product_links = []

for x in range(1, 2):  #Modify the number of pages here 
    r = requests.get(f'https://myanimelist.net/anime/genre/2/Adventure?page={x}')  #follow the similar approach for all the Genres
    soup = BeautifulSoup(r.text, 'html.parser')
    anime_links = soup.find_all('a', class_='link-title')

    for link in anime_links:
        title = link.text
        url = link['href']
        product_links.append((title, url))

anime_data = []

for title, url in product_links:
    anime_info = extract_anime_info(url)
    anime_info['Title'] = title
    anime_data.append(anime_info)

df = pd.DataFrame(anime_data)

columns_order = ['Title', 'Synonyms', 'Japanese', 'English', 'Type', 'Episodes', 'Status', 'Aired', 
                 'Premiered', 'Broadcast', 'Producers', 'Licensors', 'Studios', 'Source', 'Genres', 
                 'Themes', 'Demographic', 'Duration', 'Rating', 'Score', 'Ranked', 'Popularity', 
                 'Members', 'Favorites']

df = df.reindex(columns=columns_order)

In [4]:
df.to_csv('anime_adventure.csv')

In [5]:
df

,Title,Synonyms,Japanese,English,Type,Episodes,Status,Aired,Premiered,Broadcast,...,Genres,Themes,Demographic,Duration,Rating,Score,Ranked,Popularity,Members,Favorites
0,Fullmetal Alchemist: Brotherhood,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",鋼の錬金術師 FULLMETAL ALCHEMIST,Fullmetal Alchemist: Brotherhood,TV,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,Sundays at 17:00 (JST),...,"Action Action, Adventure Adventure, Drama Dram...",NaN,Shounen Shounen,24 min. per ep.,R - 17+ (violence & profanity),"9.09 1 (scored by 2106269 2,106,269 users) 1 i...",#2 2 2 based on the top anime page. Please not...,#3,"3,321,679","224,722"
1,Sword Art Online,"S.A.O, SAO",ソードアート・オンライン,Sword Art Online,TV,25,Finished Airing,"Jul 8, 2012 to Dec 23, 2012",Summer 2012,Sundays at 00:00 (JST),...,"Action Action, Adventure Adventure, Fantasy Fa...","Love Polygon Love Polygon, Video Game Video Game",NaN,23 min. per ep.,PG-13 - Teens 13 or older,"7.20 1 (scored by 2137462 2,137,462 users) 1 i...",#3183 2 2 based on the top anime page. Please ...,#5,"3,055,714","68,496"
2,Naruto,NARUTO,ナルト,Naruto,TV,220,Finished Airing,"Oct 3, 2002 to Feb 8, 2007",Fall 2002,Thursdays at 19:30 (JST),...,"Action Action, Adventure Adventure, Fantasy Fa...",NaN,Shounen Shounen,23 min. per ep.,PG-13 - Teens 13 or older,"7.99 1 (scored by 1960795 1,960,795 users) 1 i...",#634 2 2 based on the top anime page. Please n...,#8,"2,834,094","79,513"
3,Hunter x Hunter (2011),HxH (2011),HUNTER×HUNTER（ハンター×ハンター）,Hunter x Hunter,TV,148,Finished Airing,"Oct 2, 2011 to Sep 24, 2014",Fall 2011,Sundays at 10:55 (JST),...,"Action Action, Adventure Adventure, Fantasy Fa...",NaN,Shounen Shounen,23 min. per ep.,PG-13 - Teens 13 or older,"9.04 1 (scored by 1746028 1,746,028 users) 1 i...",#7 2 2 based on the top anime page. Please not...,#9,"2,806,664","208,992"
4,Naruto: Shippuuden,Naruto Hurricane Chronicles,-ナルト- 疾風伝,Naruto Shippuden,TV,500,Finished Airing,"Feb 15, 2007 to Mar 23, 2017",Winter 2007,Thursdays at 19:30 (JST),...,"Action Action, Adventure Adventure, Fantasy Fa...",NaN,Shounen Shounen,23 min. per ep.,PG-13 - Teens 13 or older,"8.27 1 (scored by 1645506 1,645,506 users) 1 i...",#290 2 2 based on the top anime page. Please n...,#16,"2,465,401","112,007"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Gate: Jieitai Kanochi nite, Kaku Tatakaeri Part 2","Gate: Jieitai Kanochi nite, Kaku Tatakaeri 2nd...",GATE（ゲート）自衛隊 彼の地にて、斯く戦えり 第2クール,GATE Part 2,TV,12,Finished Airing,"Jan 9, 2016 to Mar 26, 2016",Winter 2016,Saturdays at 01:05 (JST),...,"Action Action, Adventure Adventure, Fantasy Fa...","Isekai Isekai, Military Military",NaN,23 min. per ep.,R - 17+ (violence & profanity),"7.72 1 (scored by 338904 338,904 users) 1 indi...",#1164 2 2 based on the top anime page. Please ...,#394,"552,163","1,890"
96,Itai no wa Iya nanode Bougyoryoku ni Kyokufuri...,"I hate being in pain, so I think I'll make a f...",痛いのは嫌なので防御力に極振りしたいと思います。,"BOFURI: I Don't Want to Get Hurt, so I'll Max ...",TV,12,Finished Airing,"Jan 8, 2020 to Mar 25, 2020",Winter 2020,Wednesdays at 22:00 (JST),...,"Action Action, Adventure Adventure, Comedy Com...",NaN,NaN,23 min. per ep.,PG-13 - Teens 13 or older,"7.53 1 (scored by 297164 297,164 users) 1 indi...",#1706 2 2 based on the top anime page. Please ...,#412,"530,469","3,713"
97,Bleach: Sennen Kessen-hen,Bleach: Thousand-Year Blood War Arc,BLEACH 千年血戦篇,Bleach: Thousand-Year Blood War,TV,13,Finished Airing,"Oct 11, 2022 to Dec 27, 2022",Fall 2022,Tuesdays at 00:00 (JST),...,"Action Action, Adventure Adventure, Fantasy Fa...",NaN,Shounen Shounen,24 min. per ep.,R - 17+ (violence & profanity),"9.02 1 (scored by 272475 272,475 users) 1 indi...",#9 2 2 based on the top anime page. Please not...,#413,"529,327","20,248"
98,Ore dake Level Up na Ken,"Na Honjaman Level Up, 나 혼자만 레벨업, I Level Up Alone",俺だけレベルアップな件,Solo Leveling,TV,12,Currently Airing,"Jan 7, 2024 to Mar 31, 2024",Winter 2024,Sundays at 00:00 (JST),...,"